In [1]:
import sys
sys.path.append('..')

from datetime import datetime

import brasa

In [5]:
dfs = brasa.get_marketdata("b3-bvbg028", refdate=datetime(2023, 5, 12))

In [4]:
dfs["future_contracts"]

,creation_date,trade_date,security_id,security_proprietary,instrument_asset,instrument_asset_description,instrument_market,instrument_segment,instrument_description,security_category,...,contract_multiplier,asset_settlement_indicator,allocation_lot_size,trading_currency,underlying_security_id,underlying_security_proprietary,withdrawal_days,working_days,calendar_days,instrument_type
0,2023-05-12,2023-05-12,200000356461,8,DAP,Cupom de IPCA,2,5,CUPOM DE IPCA,0,...,0.000250000,1.000000000,1,BRL,9800003,8,504,501,734,FutrCtrctsInf
1,2023-05-12,2023-05-12,200001110882,8,BRI,Índice Brasil 50,2,5,INDICE BRASIL50,0,...,10.000000000,1.000000000,1,BRL,9800367,8,56,56,81,FutrCtrctsInf
2,2023-05-12,2023-05-12,200001110881,8,ICF,Café Arábica 4/5,2,4,CAFE ARABICA,0,...,100.000000000,1.000000000,1,USD,9800375,8,405,404,586,FutrCtrctsInf
3,2023-05-12,2023-05-12,200001114036,8,OZ1,Ouro,2,6,OURO 250 GRAMAS,18,...,250.000000000,1.000000000,1,BRL,10000003,8,13,13,19,FutrCtrctsInf
4,2023-05-12,2023-05-12,200001114035,8,WEU,Minicontrato de Euro,2,5,EURO MINI,0,...,10.000000000,1000.000000000,1,BRL,100000190926,8,14,14,20,FutrCtrctsInf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,2023-05-12,2023-05-12,200000710644,8,SJC,Minicontrato de Soja do CME Group,2,4,SOJA CME LIQFIN,0,...,450.000000000,1.000000000,1,USD,9800870,8,33,33,48,FutrCtrctsInf
511,2023-05-12,2023-05-12,200001087933,8,SCS,Swap Cambial de OC1,5,5,SWAP COMPROM.,20,...,50000.000000000,1.000000000,1,BRL,9800854,8,79,79,112,FutrCtrctsInf
512,2023-05-12,2023-05-12,200001093259,8,DAP,Cupom de IPCA,2,5,CUPOM DE IPCA,0,...,0.000250000,1.000000000,1,BRL,9800003,8,23,23,34,FutrCtrctsInf
513,2023-05-12,2023-05-12,200001093952,8,MIX,BVMF de Índice MICEX,2,5,INDICE MICEX,0,...,4.500000000,1.000000000,1,BRL,9800789,8,88,88,126,FutrCtrctsInf


In [3]:
import pickle
import hashlib
from datetime import datetime

In [13]:
sample_dict = {"refdate": datetime.now().date()}
t = tuple(sorted(sample_dict.items(), key=lambda x: x[0]))
t = ("b3-futures-settlement-prices", t)
hashlib.md5(pickle.dumps(t)).hexdigest()

'6b9115bafead78af0c8bbebf80b94c3f'

In [25]:
import yaml

In [32]:
yaml.safe_load(yaml.dump(sample_dict))

{'refdate': datetime.datetime(2023, 5, 15, 17, 47, 5, 405754)}

In [7]:
import sys
sys.path.append('..')

import pandas as pd

from brasa.templates import read_marketdata, download_marketdata

from lxml import etree

In [2]:
dest = download_marketdata("AjustesDiarios", refdate=datetime(2023, 5, 10))

In [14]:
tree = etree.parse(dest, etree.HTMLParser())
tree.xpath(f"//input[@id='dData1']")[0].attrib["value"]

'10/05/2023'

In [21]:
x = pd.read_html(dest, attrs=dict(id="tblDadosAjustes"), decimal=",", thousands=".", encoding="latin1")
df = x[0]

In [24]:
df.head()

,Mercadoria,Vencimento,Preï¿½o de ajuste anterior,Preï¿½o de ajuste Atual,Variaï¿½ï¿½o,Valor do ajuste por contrato (R$)
0,ABEVO - Contrato Futuro de ABEV3,K23,14.55,14.45,-0.1,0.10
1,NaN,M23,14.67,14.57,-0.1,0.10
2,AFS - Rande da ï¿½frica do Sul (em USD),M23,18634.80,18904.10,269.3,707.25
3,NaN,N23,18686.30,18959.80,273.5,718.28
4,NaN,Q23,18740.40,19019.60,279.2,733.25


In [27]:
import numpy as np


df.loc[1, "Mercadoria"] is np.nan

True

In [32]:
for ix in range(df.shape[0]):
    if df.loc[ix, "Mercadoria"] is not np.nan:
        last_name = df.loc[ix, "Mercadoria"]
    df.loc[ix, "Mercadoria"] = last_name
df.loc[:, "Mercadoria"] = df.loc[:, "Mercadoria"].str.extract(r"^(\w+)")[0]
df.Mercadoria

0      ABEVO
1      ABEVO
2        AFS
3        AFS
4        AFS
       ...  
575      ZAR
576      ZAR
577      ZAR
578      ZAR
579      ZAR
Name: Mercadoria, Length: 580, dtype: object

In [38]:
df["symbol"] = df.Mercadoria + df.Vencimento

In [39]:
df

,Mercadoria,Vencimento,Preï¿½o de ajuste anterior,Preï¿½o de ajuste Atual,Variaï¿½ï¿½o,Valor do ajuste por contrato (R$),symbol
0,ABEVO,K23,14.550,14.450,-0.100,0.10,ABEVOK23
1,ABEVO,M23,14.670,14.570,-0.100,0.10,ABEVOM23
2,AFS,M23,18634.800,18904.100,269.300,707.25,AFSM23
3,AFS,N23,18686.300,18959.800,273.500,718.28,AFSN23
4,AFS,Q23,18740.400,19019.600,279.200,733.25,AFSQ23
...,...,...,...,...,...,...,...
575,ZAR,M23,2693.591,2632.427,-61.164,2140.74,ZARM23
576,ZAR,N23,2701.409,2639.503,-61.906,2166.71,ZARN23
577,ZAR,Q23,2708.160,2645.384,-62.776,2197.16,ZARQ23
578,ZAR,U23,2716.019,2652.861,-63.158,2210.53,ZARU23


In [40]:
f = "c:\\Users\\wilso\\Dev\\python\\brasa\\notebooks\\.brasa-cache\\NegociosIntraday\\downloads\\20230514135448108822_65be6b3b8afd13e6c72fafe76c1d199d.txt"
df = pd.read_csv(f, sep=";", decimal=",")
display(df.head())
df.dtypes

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2023-05-10,BGIK23,0,264.5,5,90000000,250,1.0,2023-05-10,85,107
1,2023-05-10,BGIK23,0,264.5,1,90000000,260,1.0,2023-05-10,85,107
2,2023-05-10,BGIK23,0,264.5,1,90000000,270,1.0,2023-05-10,85,308
3,2023-05-10,BGIK23,0,264.5,3,90000000,280,1.0,2023-05-10,85,107
4,2023-05-10,BGIK23,0,264.5,2,90000000,290,1.0,2023-05-10,120,107


DataReferencia                  object
CodigoInstrumento               object
AcaoAtualizacao                  int64
PrecoNegocio                   float64
QuantidadeNegociada              int64
HoraFechamento                   int64
CodigoIdentificadorNegocio       int64
TipoSessaoPregao               float64
DataNegocio                     object
CodigoParticipanteComprador      int64
CodigoParticipanteVendedor       int64
dtype: object

In [43]:
df.CodigoInstrumento.unique()

array(['BGIK23', 'ISPM23', 'WINQ23', ..., 'DOLM23C005500', 'MDNE3T',
       'IDIF24P382200'], dtype=object)

In [32]:
download_marketdata("NegociosIntraday", refdate=datetime(2023, 5, 10))

['c:\\Users\\wilso\\Dev\\python\\brasa\\notebooks\\.brasa-cache\\NegociosIntraday\\downloads\\20230514135448108822_65be6b3b8afd13e6c72fafe76c1d199d.txt']

In [35]:
dest = "c:\\Users\\wilso\\Dev\\python\\brasa\\notebooks\\.brasa-cache\\NegociosIntraday\\downloads\\20230514135448108822_65be6b3b8afd13e6c72fafe76c1d199d.txt"
df = read_marketdata("NegociosIntraday", dest, parse_fields=True)
df

,RptDt,TckrSymb,UpdActn,GrssTradAmt,TradQty,NtryTm,TradId,TradgSsnId,TradDt,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2023-05-10,BGIK23,0,264.5,5,1900-01-01 09:00:00,250,1,2023-05-10,85,107
1,2023-05-10,BGIK23,0,264.5,1,1900-01-01 09:00:00,260,1,2023-05-10,85,107
2,2023-05-10,BGIK23,0,264.5,1,1900-01-01 09:00:00,270,1,2023-05-10,85,308
3,2023-05-10,BGIK23,0,264.5,3,1900-01-01 09:00:00,280,1,2023-05-10,85,107
4,2023-05-10,BGIK23,0,264.5,2,1900-01-01 09:00:00,290,1,2023-05-10,120,107
...,...,...,...,...,...,...,...,...,...,...,...
8997880,2023-05-10,WINM23,0,108885.0,5,1900-01-01 18:04:49,42006270,1,2023-05-10,85,3
8997881,2023-05-10,WINM23,0,108885.0,20,1900-01-01 18:04:49,42006280,1,2023-05-10,3,3
8997882,2023-05-10,WINM23,0,108885.0,5,1900-01-01 18:04:49,42006290,1,2023-05-10,3,3
8997883,2023-05-10,WINM23,0,108885.0,1,1900-01-01 18:04:49,42006300,1,2023-05-10,15,3


In [31]:
df.dtypes

RptDt                        datetime64[ns]
TckrSymb                             object
UpdActn                               int64
IntrstRateTermTrad                  float64
TradQty                               int64
NtryTm                       datetime64[ns]
TradId                                int64
TradgSsnId                            int64
TradDt                       datetime64[ns]
Mkt                                   int64
CodigoParticipanteDoador              int64
CodigoParticipanteTomador             int64
dtype: object

In [2]:
dest = "../data/NegociosBalcao.csv"
df = read_marketdata("NegociosBalcao", dest, parse_fields=True)
df

,InstrumentoFinanceiro,Emissor,CodigoIF,QuantidadeNegociada,PrecoNegocio,Volume,TaxaNegocio,OrigemNegocio,HorarioNegocio,DataNegocio,CodNegocio
0,DEB,USINATERMPAMPASUL,UTPS22,75,1089.298177,81697.36,6.88,Cetip Trader,1900-01-01 19:03:31,2022-06-03,#88847385
1,DEB,STAGENEBRA,MSGT23,200,1029.810262,205962.05,6.77,Cetip Trader,1900-01-01 19:02:59,2022-06-03,#88847384
2,DEB,STAGENEBRA,MSGT23,100,1029.810262,102981.03,6.77,Cetip Trader,1900-01-01 19:02:23,2022-06-03,#88847383
3,DEB,STAGENEBRA,MSGT23,500,1029.810262,514905.13,6.77,Cetip Trader,1900-01-01 19:02:06,2022-06-03,#88847382
4,DEB,STAGENEBRA,MSGT23,40,1029.810262,41192.41,6.77,Cetip Trader,1900-01-01 19:01:07,2022-06-03,#88847380
5,DEB,ENEVA,ENEV26,71,1029.585516,73100.57,6.27,Cetip Trader,1900-01-01 19:01:07,2022-06-03,#88847381
6,DEB,STAGENEBRA,MSGT23,25,1029.810262,25745.26,6.77,Cetip Trader,1900-01-01 19:01:06,2022-06-03,#88847378
7,DEB,STAGENEBRA,MSGT23,50,1029.810262,51490.51,6.77,Cetip Trader,1900-01-01 19:01:06,2022-06-03,#88847377


In [5]:
df.dtypes

InstrumentoFinanceiro            object
Emissor                          object
CodigoIF                         object
QuantidadeNegociada               int64
PrecoNegocio                    float64
Volume                          float64
TaxaNegocio                     float64
OrigemNegocio                    object
HorarioNegocio           datetime64[ns]
DataNegocio              datetime64[ns]
CodNegocio                       object
dtype: object

In [51]:
dt = datetime.strptime(df["HorarioNegocio"][0], "%H:%M:%S")


In [54]:
# datetime change the year
dt.replace(year=2021, month=1, day=1)

datetime.datetime(2021, 1, 1, 19, 3, 31)

In [42]:
import requests
import base64
import json
import binascii

In [43]:
args = {
    "tradingName": "ABEV",
    "language": "pt-br",
    "pageNumber": 1,
    "pageSize": 9999,
}
params = json.dumps(args).replace(" ", "").strip()
print(params)
params_enc = binascii.b2a_base64(bytes(params, "utf8"), newline=False).decode("utf8").strip()

# params_enc = "eyJ0cmFkaW5nTmFtZSI6IkFCRVYiLCJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjk5OTl9"

url = f"https://sistemaswebb3-listados.b3.com.br/listedCompaniesProxy/CompanyCall/GetListedCashDividends/{params_enc}"

requests.get(url, verify=True)

{"tradingName":"ABEV","language":"pt-br","pageNumber":1,"pageSize":9999}


<Response [200]>

In [36]:
print(params_enc)

eyJ0cmFkaW5nTmFtZSI6IkFCRVYiLCJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwi
cGFnZVNpemUiOjk5OTl9



In [30]:
print("eyJ0cmFkaW5nTmFtZSI6IkFCRVYiLCJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjk5OTl9")

eyJ0cmFkaW5nTmFtZSI6IkFCRVYiLCJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjk5OTl9


In [41]:
import binascii


binascii.b2a_base64(bytes(params, "utf8"), newline=False).decode("utf8").strip()

'eyJ0cmFkaW5nTmFtZSI6IkFCRVYiLCJsYW5ndWFnZSI6InB0LWJyIiwicGFnZU51bWJlciI6MSwicGFnZVNpemUiOjk5OTl9'

In [1]:
import json
import pandas as pd

In [2]:
with open('../data/b3-cdi.json') as fp:
    data = json.loads(fp.read())

In [3]:
data.keys()

dict_keys(['taxa', 'dataTaxa', 'indice', 'dataIndice'])

In [4]:
data

{'taxa': '11,65',
 'dataTaxa': '14/04/2022',
 'indice': '34.679,17',
 'dataIndice': '18/04/2022'}

In [9]:
pd.DataFrame(data, index=[0])

,taxa,dataTaxa,indice,dataIndice
0,"11,65",14/04/2022,"34.679,17",18/04/2022


In [37]:
data['header']

{'update': '2022-01-25', 'startMonth': 1, 'endMonth': 4, 'year': 2022}

In [6]:
df = pd.DataFrame(data['results'])

In [7]:
df

,company,spotlight,code,indexes
0,3M,DRN,MMMC34,BDRX
1,3R PETROLEUM,ON NM,RRRP3,"IBOV,IBRA,IBXX,IGCT,IGCX,IGNM,ITAG,SMLL"
2,3TENTOS,ON NM,TTEN3,"IBRA,ICON,IGCT,IGCX,IGNM,ITAG,SMLL"
3,AB INBEV,DRN,ABUD34,BDRX
4,ABB LTD,DRN,A1BB34,BDRX
...,...,...,...,...
647,WIX.COM LTD,DRN,W1IX34,BDRX
648,WIZ S.A.,ON NM,WIZS3,"GPTW,IBRA,IDIV,IGCT,IGCX,IGNM,ITAG,SMLL"
649,YDUQS PART,ON NM,YDUQ3,"IBOV,IBRA,IBXX,ICON,IGCT,IGCX,IGNM,ITAG,SMLL"
650,ZOETIS INC,DRN,Z1TS34,BDRX


In [9]:
len(df['code'].unique())

652

In [12]:
df[df['indexes'] == 'BDRX'].shape

(257, 4)

In [13]:
df[df['indexes'] != 'BDRX'].shape

(395, 4)

In [14]:
df[df['indexes'] != 'BDRX']

,company,spotlight,code,indexes
1,3R PETROLEUM,ON NM,RRRP3,"IBOV,IBRA,IBXX,IGCT,IGCX,IGNM,ITAG,SMLL"
2,3TENTOS,ON NM,TTEN3,"IBRA,ICON,IGCT,IGCX,IGNM,ITAG,SMLL"
7,ABC BRASIL,PN N2,ABCB4,"IBRA,IDIV,IFNC,IGCT,IGCX,ITAG,SMLL"
14,AERIS,ON NM,AERI3,"GPTW,IBRA,IGCT,IGCX,IGNM,INDX,ITAG,SMLL"
15,AES BRASIL,ON NM,AESB3,"IBRA,IEEX,IGCT,IGCX,IGNM,ISEE,ITAG,SMLL,UTIL"
...,...,...,...,...
642,WEG,ON NM,WEGE3,"IBOV,IBRA,IBXL,IBXX,ICO2,IGCT,IGCX,IGNM,INDX,I..."
645,WESTWING,ON NM,WEST3,"IBRA,IGCT,IGCX,IGNM,ITAG,SMLL"
646,WILSON SONS,ON NM,PORT3,"IGCX,IGNM,ITAG"
648,WIZ S.A.,ON NM,WIZS3,"GPTW,IBRA,IDIV,IGCT,IGCX,IGNM,ITAG,SMLL"


In [16]:
df['indexes'].str.split(',')

0                                                 [BDRX]
1       [IBOV, IBRA, IBXX, IGCT, IGCX, IGNM, ITAG, SMLL]
2             [IBRA, ICON, IGCT, IGCX, IGNM, ITAG, SMLL]
3                                                 [BDRX]
4                                                 [BDRX]
                             ...                        
647                                               [BDRX]
648     [GPTW, IBRA, IDIV, IGCT, IGCX, IGNM, ITAG, SMLL]
649    [IBOV, IBRA, IBXX, ICON, IGCT, IGCX, IGNM, ITA...
650                                               [BDRX]
651                                               [BDRX]
Name: indexes, Length: 652, dtype: object

In [34]:
def _(dfx):
    indexes = dfx['indexes'].str.split(',').explode()
    return pd.DataFrame({
        'company': dfx['company'],
        'spotlight': dfx['spotlight'],
        'code': dfx['code'],
        'indexes': indexes,
    })

dfr = df.groupby(['company', 'spotlight', 'code']).apply(_).reset_index(drop=True)

In [40]:
dfr['refdate'] = data['header']['update']
dfr['duration_start_month'] = data['header']['startMonth']
dfr['duration_end_month'] = data['header']['endMonth']
dfr['duration_year'] = data['header']['year']
dfr

,company,spotlight,code,indexes,refdate,duration_start_month,duration_end_month,duration_year
0,3M,DRN,MMMC34,BDRX,2022-01-25,1,4,2022
1,3R PETROLEUM,ON NM,RRRP3,IBOV,2022-01-25,1,4,2022
2,3R PETROLEUM,ON NM,RRRP3,IBRA,2022-01-25,1,4,2022
3,3R PETROLEUM,ON NM,RRRP3,IBXX,2022-01-25,1,4,2022
4,3R PETROLEUM,ON NM,RRRP3,IGCT,2022-01-25,1,4,2022
...,...,...,...,...,...,...,...,...
2468,YDUQS PART,ON NM,YDUQ3,IGNM,2022-01-25,1,4,2022
2469,YDUQS PART,ON NM,YDUQ3,ITAG,2022-01-25,1,4,2022
2470,YDUQS PART,ON NM,YDUQ3,SMLL,2022-01-25,1,4,2022
2471,ZOETIS INC,DRN,Z1TS34,BDRX,2022-01-25,1,4,2022


In [1]:
import sys
sys.path.insert(0, '..')

from kyd.parsers.anbima import AnbimaDebentures

deb = AnbimaDebentures('../data/deb_2021-04-20.txt')


In [2]:
df = deb.data

In [3]:
df

,symbol,name,maturity_date,underlying,bid_yield,ask_yield,ref_yield,price,perc_price_par,duration,perc_reune,ref_ntnb,refdate
0,ALSCA0,ALIANSCE SONAE SHOPPING CENTER S/A (*),2023-09-15,110% do DI,125.7619,122.785,123.8620,NaN,NaN,NaN,NaN,None,2021-04-20
1,AMPLA0,AMPLA ENERGIA E SERVIÇOS S/A (*),2024-03-15,108% do DI,127.2385,120.4444,124.1191,978.146777,97.550163,564.82,NaN,None,2021-04-20
2,BRML17,BR MALLS PARTICIPAÇÕES S/A (*),2025-03-11,"107,5% do DI",124.6151,121.4285,123.1240,9626.591287,95.991375,863.50,NaN,None,2021-04-20
3,BSA312,"B3 S/A - BRASIL, BOLSA, BALCÃO (*) (**)",2022-05-03,"102,8% do DI",129.7504,122.4282,127.9462,9963.089620,98.690110,254.61,NaN,None,2021-04-20
4,CBRDA6,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO (*) (**),2021-09-10,"162,71% do DI",132.1536,124,130.1143,1009.549332,100.507274,100.00,NaN,None,2021-04-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,VLIO11,VLI OPERACOES PORTUARIAS S/A,2021-08-15,"IPCA + 6,0500%",0.9068,0.3652,0.7968,1270.884952,101.666888,82.00,NaN,2022-08-15,2021-04-20
526,VNTT11,VENTOS DE SÃO TITO HOLDING S/A,2028-06-15,"IPCA + 9,2360%",8.0168,6.673,7.4670,1034.560334,106.054166,916.77,NaN,2025-05-15,2021-04-20
527,VOES16,CONC. DE ROD. DO OESTE DE SP - VIAOESTE S/A (*),2021-11-15,"IPCA + 6,2959%",1.6093,0.7842,1.2885,1257.677653,102.723202,140.36,NaN,2022-08-15,2021-04-20
528,VPLT12,VIAPAULISTA S.A.,2027-06-15,"IPCA + 3,9407%",4.3572,4.0585,4.1606,1008.504632,99.356190,769.31,NaN,2024-08-15,2021-04-20
